In [1]:
                     from korbash_lib import Puller, PlotDisplayer, Slider
import time
import numpy as np
import pandas as pd
import datetime
from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.layouts import gridplot, layout
from jupyter_ui_poll import ui_events
import ipywidgets
import matplotlib
import scipy
from scipy import stats
import random
import os
output_notebook()
pl = Puller() #simulate=True
random.seed()

# инициализация запускать только один раз

Loading BokehJS ...

[(21, 90113195), (21, 90113196), (21, 90113197)]


In [2]:
#pl.ms.motorM.MoveTo(30);

In [3]:
pl.ms.StopButton() # кнопки для остоновки моторчиков в нештатной ситуации

GridspecLayout(children=(Button(description='Stop All Motors', layout=Layout(grid_area='widget001'), style=But…

In [18]:
pl.ms.MoveToStart(zapas=10) # сдвиг моторчиков на начальные позиции для растяжки
pl.ms.motorM.MoveTo(17);
while pl.ms.IsInMotion():
    pass

In [11]:
pl.tg.SetZeroWeight(15) # калибровка тензодатчика

0.8139785419472824

In [12]:
x0=pl.SetH_avto() # измерение высоты молибдена

SetW finish value= 49.927216677988824
SetH_podgon [ 7.         49.92017852 14.3279295 ]
podgon a= 5.236556661884956   b= 50.02770690022227   x0= 14.594890208227248
podgon a= 5.705016893211002   b= 50.03834541463348   x0= 14.745944431868075
data saved
data saved


In [13]:
pl.SetW(20, dw=0.1, tau=2,quiet=False); # натяжка волокна

50.20568114090702    30.205681140907018    -0.06990885975811957
22.54868368412845    2.5486836841284486    -0.005898743663828299
17.973101285515472    -2.0268987144845276    0.004691110169434728
19.362880675210356    -0.6371193247896443    0.0014745665001934672
20.12863078594924    0.12863078594924104    -0.0002977066311980596
20.22655795000855    0.2265579500085515    -0.0005243519548640543
20.143735265232333    0.14373526523233338    -0.00033266485375875156
20.01005936382057    0.010059363820570155    -2.3281668481753164e-05
SetW finish value= 20.03520464936314


In [14]:
pl.ms.motorM.MoveTo(pl.ms.motorM.position_max-2); # опускание молибдена вниз перед разогревом

In [15]:
pl.tg.SetZeroWeight(10,20) # калибровка тензодатчика после разогрева молибдена

96.51513847094657

In [7]:
x0=14.81950332231099
pl.ms.x0=x0

In [19]:
while pl.ms.IsInMotion():
    pass
dp=PlotDisplayer()
dp2=PlotDisplayer()
dp2.CreateMaket(mainParam= 'time',obr_coff=pd.DataFrame({
    'time': pl.times,
    'kof': pl.trueKmas
}))
dp.CreateMaket(mainParam='time',power=pl.data[['time','power']],
                    tension=pl.data[['time','tension','tensionWgl', 'tensionEXPgl']],
                  # motors=pl.data[['time','motorR','motorL','motorM']]
                  # v_and_a=pl.data[['time','vL','vR','aL','aR']]
              )
sl=Slider()
sl.NewSl('v',min=0.1,max=9,value=3,step=0.1)
sl.NewSl('a',min=0.1,max=9,value=7,step=0.1)
sl.NewSl('dvKof', max=3,value=0,step=0.01)
sl.NewSl('dv', max=0.5,value=0.001,step=0.0001)
sl.NewSl('window',min=50, max=10000,value=100, step=50)
sl.NewSl('molibdenH',min=-1.6, max=2.0,value=2.0,step=0.01)
sl.NewSl('obrSv',min=0, max=10,value=1,step=0.1)
sl.NewSl('T',min=0, max=200,value=10,step=0.1)
sl.NewBtn('molibden','up molibden','doun molibden')
sl.NewBtn('end','end','end2','end3','vse','bolshe ne jmi','nu pravda ne nado', 'samiy umniy da?')
sl.NewBtn('start dv','start dv','stop dv')
sl.NewBtn('start stabili','start st','stop st')
sl.NewTt('x')
sl.NewTt('L')
sl.NewTt('R')
sl.NewTt('dv')

In [20]:
pl.MotorsControlStart()
t0 = time.time()
t = t0
dp.Show()
# dp2.Show()
sl.Display()
pl.Clear()

with ui_events() as poll:
    while True:
        poll(10)
        t = time.time()     
        if pl.PulMotorsControl(sl.Sl['molibdenH'](),sl.Sl['T'](),
                                    upFl=sl.BtnFl['molibden'], stFl=sl.BtnFl['end'],
                                    vsFl=sl.BtnFl['start stabili'], dhKof=0.5, ah=9,obrKof=sl.Sl['obrSv']()*0.0001
                                 )==-1:
            break
        x=pl.data['x'].iloc[-1]
        pl.v=sl.Sl['v']()
        pl.a=sl.Sl['a']()
        T=sl.Sl['T']()
        ww=int(sl.Sl['window']())
        sl.ChangeValueTt('x',x)
        sl.ChangeValueTt('L',pl.ms.L_x(x))
        sl.ChangeValueTt('R',pl.ms.R_x(x))
        sl.ChangeValueTt('dv',pl.dv)    
        dp.Apdate(for_all=pl.data.iloc[-ww:])
#         dp2.Apdate(for_all=pd.DataFrame({
#                     'time': pl.times,
#                     'kof': pl.trueKmas
#                 }))
        push_notebook()    
        time.sleep(0.04)
print('ura')
pl.Save()
with ui_events() as poll:
    while sl.BtnFl['end']<3:
        poll(10)
        pl.SetW(2,dw=1,tau=0.5)
print('vse')

xMax= 38.44613234964727 L_x(xMax)= 14.700000000000001   x= -3.69702976854569e-06   L= 5.364047276454983 xEnd= -3.69702976854569e-06 lEnd= 4.721193254764884
ura
data saved
data saved
vse


In [11]:
pl.ms.End() # растановка моторчиков на нули (запускать перед отключением питания)

ending
end


In [32]:
pl.Save() # сохранение данных

data saved
